In [1]:
import cv2
import numpy as np
import dlib

In [13]:
#Importing Image
img = cv2.imread("img4.jpg")
# print(img.shape)
if img.shape[0] > 500 and img.shape[1] > 500:
    img = cv2.resize(img, (0,0), None, fx = 0.5, fy = 0.5)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:

face_detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

def empty(a):
    pass

cv2.namedWindow("LipColor")
cv2.resizeWindow("LipColor", 640, 240)
cv2.createTrackbar("Blue",'LipColor',0,255,empty)
cv2.createTrackbar("Green",'LipColor',0,255,empty)
cv2.createTrackbar("Red",'LipColor',0,255,empty)



def createBoundBox(img,points,teeth_points, scale=2, masked=False, cropped=True):
    #converting lists to numpy array
    points = np.array(points)
    teeth_points = np.array(teeth_points)

    if masked:
        #creating mask only for lips
        mask = np.zeros_like(img)
        mask = cv2.fillPoly(mask,[points], (255,255,255))
        mask = cv2.fillPoly(mask,[teeth_points], (0,0,0))
        img = cv2.bitwise_and(img, mask)

    if cropped:
        #getting starting (x,y) coordinate, width and height of image
        x,y,w,h = cv2.boundingRect(points)
        crop = img[y:y+h, x:x+w]
        crop = cv2.resize(crop,(0,0),fx=scale, fy=scale)
        return crop
    else:
        return mask

while True:
    img = cv2.imread("./assets/img1.jpg")
    if img.shape[0] > 500 and img.shape[1] > 500:
        img = cv2.resize(img, (0,0), None, fx = 0.5, fy = 0.5)

    img_copy = img.copy()
    img_bw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector(img_bw)
    # face_points = []
    for face in faces:
        x1, y1 = face.left(), face.top()
        x2, y2 = face.right(), face.bottom()
        landmarks = predictor(img_bw, face)
        x = [landmarks.part(i).x for i in range(68)]
        y = [landmarks.part(i).y for i in range(68)]
        face_points = [[i,j] for i,j in zip(x,y)]  

    #Cropping Lips section
    lips = createBoundBox(img, face_points[48:61], face_points[60:69], masked=True, cropped=False)

    #Coloring Lips
    lip_color_img = np.zeros_like(lips)
    b = cv2.getTrackbarPos('Blue',"LipColor")
    g = cv2.getTrackbarPos('Green',"LipColor")
    r = cv2.getTrackbarPos('Red',"LipColor")

    lip_color_img[:] = b,g,r
    lip_color_img = cv2.bitwise_and(lips, lip_color_img)
    lip_color_img = cv2.GaussianBlur(lip_color_img, (7,7),10)
    lip_color_img = cv2.addWeighted(img_copy, 1, lip_color_img,0.4,0)
    # lip_color_img = cv2.resize(lip_color_img, (0,0), fx=1.5, fy=1.5)
    # img_copy = cv2.resize(img_copy, (0,0), fx=1.5, fy=1.5)

    collage = np.hstack((img_copy,lip_color_img))
    cv2.imshow('LipColor', collage)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

In [7]:
a = [1,2,3,4,5,6]
b = [2,4,6,8,10,12]
b[-1]

12